In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/OnePieceClassificationData

/content/drive/MyDrive/OnePieceClassificationData


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
from PIL import Image
from torch.utils.data import Dataset

In [3]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = []

        for root, _, files in os.walk(root_dir):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    self.image_paths.append(os.path.join(root, file))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, img_path

In [5]:
# Data loading
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/OnePieceClassificationData/splitted/train', transform=transform, allow_empty=True)
test_dataset = CustomImageDataset('/content/drive/MyDrive/OnePieceClassificationData/splitted/test', transform=transform)

In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


# Model
model = models.regnet_x_400mf(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 18)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RegNet_X_400MF_Weights.IMAGENET1K_V1`. You can also use `weights=RegNet_X_400MF_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
 # Training loop
for epoch in range(3):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
# Evaluation
correct = 0
total = 0
predicted_label = []
image_name = []
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data,
 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predicted_label.append(predicted)
        image_name.append(images)

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
print (predicted_label)
print (image_name)

In [ ]:
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

In [ ]:
test_dataset = OnePieceDataset(images_dir=TEST_DATA_PATH, transform=transform)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()

predictions = []
image_names = []

with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)

        outputs = model(images)

        _, preds = outputs.max(1)

        predictions.extend(preds.cpu().numpy())
        image_names.extend(filenames)

predictions_df = pd.DataFrame({
    'id': image_names,
    'label': predictions
})

predictions_df.to_csv('submission.csv', index=False)

NameError: name 'OnePieceDataset' is not defined

In [ ]:
class OnePieceDataset(Dataset):
    def __init__(self, images_dir, csv_path=None, labels_json=None, transform=None):
        self.images_dir = images_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.is_train = csv_path is not None

        if self.is_train:
            import pandas as pd
            self.data = pd.read_csv(csv_path)
            self.label_map = labels_json if isinstance(labels_json, dict) else None


            for _, row in self.data.iterrows():
                relative_path = row['image_path'].replace("\\", "/")
                image_path = os.path.join(images_dir, relative_path)
                image_path = os.path.normpath(image_path)
                self.image_paths.append(image_path)
                self.labels.append(row['label'])

        else:
            self.image_paths = [
                os.path.join(images_dir, fname)
                for fname in os.listdir(images_dir)
                if os.path.isfile(os.path.join(images_dir, fname))
            ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        if self.is_train:
            label = self.labels[idx]
            return image, label
        else:
            image_name = os.path.splitext(os.path.basename(image_path))[0]
            return image, image_name

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from matplotlib import pyplot as plt

In [ ]:
from PIL import Image

In [ ]:
from torchvision import transforms

In [ ]:
#import torch.optim as optim
#from torchvision import datasets, transforms, models

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

import cv2

In [ ]:
TEST_DATA_PATH = "./splitted/test"
TRAIN_DATA_PATH = "./splitted"
CSV_PATH = "./train_annotations.csv"

In [ ]:
labels_map = {
    0: "Ace",
    1: "Akainu",
    2: "Brook",
    3: "Chopper",
    4: "Crocodile",
    5: "Franky",
    6: "Jinbei",
    7: "Kurohige",
    8: "Law",
    9: "Luffy",
    10: "Mihawk",
    11: "Nami",
    12: "Rayleigh",
    13: "Robin",
    14: "Sanji",
    15: "Shanks",
    16: "Usopp",
    17: "Zoro",
}

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = OnePieceDataset(images_dir=TRAIN_DATA_PATH, csv_path=CSV_PATH, labels_json=labels_map, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = OnePieceDataset(images_dir=TEST_DATA_PATH, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import tensorflow as tf

In [ ]:
# Функция для загрузки изображения
def load_image(image_path):
  img = cv2.imread(image_path)
  # Преобразование изображения в нужный формат (например, в тензор)
  img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
  return img_tensor

# Путь к папке с изображениями
data_dir = './splitted'

# Создаем списки для хранения путей к изображениям
train_images = []
test_images = []

# Проходим по всем файлам в папке и подпапках
for root, dirs, files in os.walk(data_dir):
  for file in files:
    if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):  # Или другие поддерживаемые форматы
      image_path = os.path.join(root, file)
      # Разделяем данные на обучающую и тестовую выборки
      if 'train' in root:
        train_images.append(image_path)
      elif 'test' in root:
        test_images.append(image_path)

# Загружаем изображения
train_data = [load_image(img_path) for img_path in train_images]
test_data = [load_image(img_path) for img_path in test_images]

In [ ]:
def plot_samples(data, labels_map, cols=3, rows=3):
    figure = plt.figure(figsize=(8, 8))
    for i in range(1, cols * rows + 1):
        sample_idx = torch.randint(len(data), size=(1,)).item()
        img, label = data[sample_idx]
        figure.add_subplot(rows, cols, i)
        if type(label) == torch.Tensor:
            plt.title(labels_map[label.item()])
        else:
            plt.title(labels_map[label])
        plt.axis("off")
        plt.imshow(img.squeeze(), cmap="gray")
    plt.show()

In [ ]:
plot_samples(train_data, labels_map, cols=3, rows=3)

ValueError: too many values to unpack (expected 2)

<Figure size 800x800 with 0 Axes>

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze().reshape(28,28)
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/OnePieceClassificationData/splitted/train/Kurohige/81.png'

In [ ]:
# Подготовка данных
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('train_data', transform=train_transform)
test_dataset = datasets.ImageFolder('test_data', transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

FileNotFoundError: [Errno 2] No such file or directory: 'train_data'

In [ ]:
# Использование предобученной модели ResNet
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 18)  # 18 классов персонажей

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 112MB/s]


In [ ]:
# Обучение модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    # ... (процесс обучения)

In [ ]:
# Предсказания на тестовом наборе
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        # ... (сохранение предсказаний в CSV-файл)